In [ ]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

X = np.array([[1, 2],
              [2, 3],
              [3, 4],
              [4, 5],
              [5, 6]])
y = np.array([1, 1, 0, 0, 0])

# Initialize AdaBoost Classifier
clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.1, random_state=42,
                         algorithm='SAMME')

# Train the model
clf.fit(X, y)

# Make predictions
y_pred = clf.predict(X)

def accuracy(y_test, y_pred):
  return np.mean(y_test == y_pred)

acc = accuracy(y, y_pred)
print(acc)

1.0


# AdaBoostWithTreeStump Pseudocode

## Class: `AdaBoostWithTreeStump`

### Attributes:
- `n_estimators`: Number of weak learners (tree stumps) to train.
- `alphas`: List of alpha weights (contribution of each weak learner to the final prediction).
- `models`: List of trained weak learners (tree stumps).

---

## Methods:

### `__init__(n_estimators=50)`
**Purpose:** Initialize the AdaBoost model with the specified number of weak learners.

1. Set `self.n_estimators` to `n_estimators`.
2. Initialize `self.alphas` as an empty list.
3. Initialize `self.models` as an empty list.

---

### `fit(X, y)`
**Purpose:** Train the AdaBoost model on the input data `X` and target labels `y`.

1. Determine the number of samples `m = X.shape[0]`.
2. Initialize sample weights:
   - `weights = array of size m, each element = 1/m`.
3. **For each estimator (from 1 to `n_estimators`)**:
   - Train a weak learner (tree stump):
     - Create a `DecisionTree` with `max_depth=1`.
     - Fit the tree stump on `X` and `y` using the current weights.
   - Predict labels for the current dataset:
     - `predictions = stump.predict(X)`.
   - Compute the weighted classification error:
     - `error = sum(weights[i] for i where predictions[i] != y[i])`.
   - Calculate the alpha value:
     - If `error == 0`:
       - `alpha = very large value (e.g., 1e10)`.
     - Otherwise:
       - `alpha = 0.5 * log((1 - error) / error)`.
   - Append the alpha value to `self.alphas`.
   - Add the trained stump to `self.models`.
   - Update the sample weights:
     - For each sample `i`:
       - If `predictions[i] == y[i]`:
         - `weights[i] *= exp(-alpha)`.
       - Else:
         - `weights[i] *= exp(alpha)`.
     - Normalize the weights to ensure they sum to 1.
   - Perform resampling based on updated weights:
     - Compute cumulative weights for the current sample set.
     - Draw `m` samples with replacement using the weights as probabilities.
     - Update `X` and `y` to the resampled dataset.

---

### `predict(X)`
**Purpose:** Predict labels for the input data `X` using the trained AdaBoost model.

1. Initialize predictions:
   - `final_predictions = array of size len(X), filled with zeros`.
2. **For each trained model (`stump`) and corresponding alpha (`alpha`)**:
   - Predict the labels using the model:
     - `predictions = stump.predict(X)`.
   - Add the weighted predictions to `final_predictions`:
     - `final_predictions += alpha * predictions`.
3. Convert the final predictions to binary labels:
   - Return `1` where `final_predictions > 0` and `0` otherwise.


In [ ]:
import numpy as np

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
  def __init__(self, max_depth=3):
    self.max_depth = max_depth
    self.root = None

  def fit(self, X, y):
    self.root = self._build_tree(X, y)

  def _build_tree(self, X, y, depth=0):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    if(n_labels == 1 or self.max_depth is not None and depth >= self.max_depth):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    best_feature, best_threshold = self._best_split(X, y)

    left_idxs , right_idxs = self._split(X[:, best_feature], best_threshold)

    left = self._build_tree(X[left_idxs, :], y[left_idxs], depth+1)
    right = self._build_tree(X[right_idxs, :], y[right_idxs], depth+1)
    return Node(best_feature, best_threshold, left, right)

  def _most_common_label(self, y):
    unique_labels, counts = np.unique(y, return_counts=True)
    return unique_labels[np.argmax(counts)]

  def _split(self, X_column, split_threshold):
    left_idxs = np.argwhere(X_column <= split_threshold).flatten()
    right_idxs = np.argwhere(X_column > split_threshold).flatten()
    return left_idxs, right_idxs

  def _best_split(self, X, y):
    best_gain, split_idx, split_threshold = float("inf"), None, None

    for feat_idx in range(X.shape[1]):
      X_Column = X[:, feat_idx]
      X_Column_sorted = np.sort(X_Column)
      thresholds = (X_Column_sorted[:-1] + X_Column_sorted[1:])/2

      for threshold in thresholds:
        gain = self._information_gain(y, X_Column, threshold)

        if gain < best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = threshold

    return split_idx, split_threshold


  def _information_gain(self, y, X_column, threshold):
    left_idxs, right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0

    n, n_l, n_r = len(y), len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    information_gain = (n_l/n)*e_l + (n_r/n)*e_r
    return information_gain
  def _entropy(self, y):
    fid3 = np.mean(y)

    if fid3 == 0 or fid3 == 1: return 0
    else: return -fid3 * np.log(fid3) - (1 - fid3) * np.log(1 - fid3)

  def predict(self, X):
    predictions = np.array([self._traverse_tree(x, self.root) for x in X])
    return predictions

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)

class AdaBoostWithTreeStump:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []

    def fit(self, X, y):
        m = X.shape[0]
        weights = np.ones(m) / m

        for _ in range(self.n_estimators):
            stump = DecisionTree(max_depth=1)
            stump.fit(X, y)

            predictions = stump.predict(X)
            error = np.sum(predictions != y) / m

            if error == 0:
                alpha = 1e10
            else:
                alpha = 0.5 * np.log((1 - error) / error)

            self.alphas.append(alpha)
            self.models.append(stump)

            new_weights = []
            for i in range(m):
                if predictions[i] == y[i]:
                    new_weight = weights[i] * np.exp(-alpha)
                else:
                    new_weight = weights[i] * np.exp(alpha)
                new_weights.append(new_weight)

            min_weight = 1e-10
            new_weights = [max(w, min_weight) for w in new_weights]
            weight_sum = np.sum(new_weights)
            weights = [w / weight_sum for w in new_weights]
            # weights /= np.sum(weights)

            cumulative_weights = np.cumsum(weights)

            random_values = np.random.rand(m)

            indices = np.searchsorted(cumulative_weights, random_values)
            X, y = X[indices], y[indices]

    def predict(self, X):
        m = len(X)
        final_predictions = np.zeros(m)

        for alpha, model in zip(self.alphas, self.models):
            predictions = model.predict(X)
            final_predictions += alpha * predictions
        return np.where(final_predictions > 0, 1, 0)


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Breast Cancer Dataset
# data = load_breast_cancer()
# X, y = data.data, data.target

X = np.array([[1, 2],
              [2, 3],
              [3, 4],
              [4, 5],
              [5, 6]])
y = np.array([1, 1, 0, 0, 0])

clf = AdaBoostWithTreeStump(n_estimators=100)
clf.fit(X, y)

y_pred = clf.predict(X)

def accuracy(y_test, y_pred):
  return np.mean(y_test == y_pred)

acc = accuracy(y, y_pred)
print(acc)

1.0
